# Two Player Quantum Lower Bound

### Description:
Compute a lower bound on the quantum value of a nonlocal game.

Calculates a lower bound on the maximum value that the specified nonlocal
game can take on in quantum mechanical settings where Alice and Bob each
have access to a d-dimensional quantum system.

This approach works by starting with a randomly-generated POVM for Bob, and
then optimizing Alice's POVM and the shared entangled state. Then Alice's
POVM and the entangled state are fixed, and Bob's POVM is optimized. And so
on, back and forth between Alice and Bob until convergence is reached.

Note that the algorithm is not guaranteed to obtain the optimal local bound
and can get stuck in local minimum values. The alleviate this, it is run
more than once to run the algorithm some pre-specified number of
times and keep the highest value obtained.

### Explanation:
The algorithm is based on the alternating projections algorithm as it can
be applied to Bell inequalities as shown in [1].

The alternating projection algorithm has also been referred to as the 
"see-saw" algorithm as it goes back and forth between the following two 
semidefinite programs:

$$
\textbf{SDP-1:} \qquad
\begin{align*}
                \text{maximize:} \quad & \sum_{(x,y \in \Sigma)} \pi(x,y)
                                         \sum_{(a,b) \in \Gamma}
                                         V(a,b|x,y)
                                         \langle B_b^y, A_a^x \rangle \\
                \text{subject to:} \quad & \sum_{a \in \Gamma_{\mathsf{A}}} =
                                           \tau, \qquad \qquad
                                           \forall x \in \Sigma_{\mathsf{A}}, \\
                                   \quad & A_a^x \in \text{Pos}(\mathcal{A}),
                                           \qquad
                                           \forall x \in \Sigma_{\mathsf{A}}, \
                                           \forall a \in \Gamma_{\mathsf{A}}, \\
                                           & \tau \in \text{D}(\mathcal{A}).
\end{align*}
$$

$$
\textbf{SDP-2:} \qquad
\begin{align*}
                \text{maximize:} \quad & \sum_{(x,y \in \Sigma)} \pi(x,y)
                                         \sum_{(a,b) \in \Gamma} V(a,b|x,y)
                                         \langle B_b^y, A_a^x \rangle \\
                \text{subject to:} \quad & \sum_{b \in \Gamma_{\mathsf{B}}} =
                                           \mathbb{I}, \qquad \qquad
                                           \forall y \in \Sigma_{\mathsf{B}}, \\
                                   \quad & B_b^y \in \text{Pos}(\mathcal{B}),
                                   \qquad \forall y \in \Sigma_{\mathsf{B}}, \
                                   \forall b \in \Gamma_{\mathsf{B}}.
\end{align*}
$$




## Examples

In [6]:
import cvxpy
import numpy as np
from toqito.nonlocal_games.two_player_quantum_lower_bound import two_player_quantum_lower_bound

### CHSH game

The CHSH game is a two-player nonlocal game with the following probability distribution and question and answer sets.

$$
\begin{equation} 
\begin{aligned} \pi(x,y) = \frac{1}{4}, \qquad (x,y) \in \Sigma_A \times \Sigma_B, \qquad \text{and} \qquad (a, b) \in \Gamma_A \times \Gamma_B, 
\end{aligned} 
\end{equation}
$$

where

$$
\begin{equation} 
\Sigma_A = \{0, 1\}, \quad \Sigma_B = \{0, 1\}, \quad \Gamma_A = \{0,1\}, \quad \text{and} \quad \Gamma_B = \{0, 1\}. 
\end{equation}
$$

Alice and Bob win the CHSH game if and only if the following equation is satisfied

$$
\begin{equation} 
a \oplus b = x \land y. 
\end{equation}
$$

Recall that $\oplus$ refers to the XOR operation.

The optimal quantum value of CHSH is $\cos(\pi/8)^2 \approx 0.8536$ where the optimal classical value is $3/4$. 

In [12]:
dim = 2
num_alice_inputs, num_alice_outputs = 2, 2
num_bob_inputs, num_bob_outputs = 2, 2
prob_mat = np.array([[1 / 4, 1 / 4], [1 / 4, 1 / 4]])
pred_mat = np.zeros(
    (num_alice_outputs, num_bob_outputs, num_alice_inputs, num_bob_inputs)
)

for a_alice in range(num_alice_outputs):
    for b_bob in range(num_bob_outputs):
        for x_alice in range(num_alice_inputs):
            for y_bob in range(num_bob_inputs):
                if np.mod(a_alice + b_bob + x_alice * y_bob, dim) == 0:
                    pred_mat[a_alice, b_bob, x_alice, y_bob] = 1

lower_bound = two_player_quantum_lower_bound(dim, prob_mat, pred_mat)

# cos(pi/8)^2 /approx 0.85356
lower_bound

0.8535533840915605

For CHSH, the lower bound approach obtained the optimal quantum value of $\cos(\pi/8)^2$. However, the optimal values are not guaranteed to be obtained by this approach.

## References

Liang, Yeong-Cherng, and Andrew C. Doherty.
"Bounds on quantum correlations in Bell-inequality experiments."
Physical Review A 75.4 (2007): 042103.
https://arxiv.org/abs/quant-ph/0608128